In [1]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
import json
import matplotlib.pyplot as plt
import time
import joblib

In [2]:
model = joblib.load("aeration_classifier.joblib")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.0 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.0 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
# Initialize data storage for plotting
timestamps = []
water_temperatures = []
ph_levels = []
turbidities = []
dissolved_oxygen_levels = []

In [4]:
# Kafka configuration
def initialize_consumer():
    kafka_topic = "water_quality"
    kafka_bootstrap_servers = ["localhost:9092"]

    # Create Kafka consumer
    consumer = KafkaConsumer(
        kafka_topic,
        bootstrap_servers=kafka_bootstrap_servers,
        value_deserializer=lambda m: json.loads(m.decode('utf-8')),
        auto_offset_reset='latest',
        enable_auto_commit=True, 
        group_id = 'water_quality_predictors'
    )
    return consumer


In [5]:
kafka_topic_pred = "water_quality_predictions"

def predict(consumer, producer):
    try:
        for message in consumer:
            # Parse the message
            sensor_data = message.value
            sensor_key = message.key
            print(f"Received: {sensor_data} Key: {sensor_key}")

            timestamp = sensor_data['timestamp']
            water_temperature = sensor_data['water_temperature']
            ph_level = sensor_data['ph_level']
            turbidity = sensor_data['turbidity']
            dissolved_oxygen_level = sensor_data['dissolved_oxygen']

            prediction = model.predict([[water_temperature, ph_level, turbidity, dissolved_oxygen_level]])
            prediction_probs = model.predict_proba([[water_temperature, ph_level, turbidity, dissolved_oxygen_level]])

            message = { 'prediction': prediction.tolist(), 'prediction_probabilities': prediction_probs.tolist(), 'timestamp': timestamp }
            
            producer.send(kafka_topic_pred, message, key = sensor_key)

            print(f"Mensaje: {message}")
            break  # Process one message at a time
    except KeyboardInterrupt:
        print("Stopped consuming messages.")
        consumer.close()
    

In [ ]:
consumer = initialize_consumer()

# Kafka configuration
kafka_bootstrap_servers = ["localhost:9092"] 

# Create Kafka producer
producer = KafkaProducer(
    bootstrap_servers=kafka_bootstrap_servers,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

try:
    while True:
        predict(consumer, producer)
except KeyboardInterrupt:
    print("Stopped.")
    consumer.close()

Received: {'timestamp': 1741191886, 'water_temperature': 29.049932041378877, 'ph_level': 8.459878014909602, 'turbidity': 48.51, 'dissolved_oxygen': 9.17} Key: b'4'
Mensaje: {'prediction': [0], 'prediction_probabilities': [[1.0, 0.0]], 'timestamp': 1741191886}
Received: {'timestamp': 1741191890, 'water_temperature': 29.224635286648784, 'ph_level': 8.119361630956142, 'turbidity': 23.75, 'dissolved_oxygen': 8.66} Key: b'1'
Mensaje: {'prediction': [0], 'prediction_probabilities': [[0.9, 0.1]], 'timestamp': 1741191890}
Received: {'timestamp': 1741191892, 'water_temperature': 28.399890548724684, 'ph_level': 8.191720743059676, 'turbidity': 15.5, 'dissolved_oxygen': 8.32} Key: b'3'
Mensaje: {'prediction': [0], 'prediction_probabilities': [[0.7, 0.3]], 'timestamp': 1741191892}
Received: {'timestamp': 1741191893, 'water_temperature': 27.509694219790763, 'ph_level': 8.14405684555988, 'turbidity': 36.85, 'dissolved_oxygen': 10.34} Key: b'4'
Mensaje: {'prediction': [0], 'prediction_probabilities': 